# Testing Set Period: 20140930-20141231


In [1]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn import linear_model

### 1. BE/ME Ratio in 2014 (same throughout the year)
The size breakpoint for year t is the median NYSE market equity at the end of June of year t. BE/ME for June of year t is the book equity for the last fiscal year end in t-1 divided by ME for December of t-1. 

In [3]:
BEME_ratio_df_2014 = pd.read_csv("/Users/yiliwang/Documents/ACR Internship/Intermediate Files/BEME_ratio_df_2014.csv", header=0)

In [35]:
norm_BEME_ratio_list = list(BEME_ratio_df_2014['norm_BEME_ratio'])# 268 companies available total

###### X1: norm_BEME_ratio_list

### 2. Last Quarter Return

Duration: 20140630 - 20140930

return = [SP(20140930) - SP(20140630)]/SP(20140630)

sample pool: companies in BEME_ratio_df['WINDCODE']

#### a) Last Quarterly Return for Individual Company

In [6]:
##### Load in all the files of Stock Price

# set current working directory to the directory storing stock price data
os.chdir('/Users/yiliwang/Documents/ACR Internship/data/')

In [7]:
files = [file for file in os.listdir( os.curdir ) if file.endswith(".csv")]

In [8]:
# Store files to dictionary using their own name
frames = {}
for each_file in files:
    frames[os.path.splitext(each_file)[0]] = pd.read_csv(each_file)

##### Extract Stock Price Data on Starting Day and End Day

In [10]:
# Obatin Close Price
start_close = [None]*len(BEME_ratio_df_2014['WINDCODE'])
end_close = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [11]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    company = BEME_ratio_df_2014['WINDCODE'][i]
    start_close[i] =float(frames[company][frames[company]['TRADE_DT'] == 20140630]['S_DQ_ADJCLOSE'])
    end_close[i] = float(frames[company][frames[company]['TRADE_DT'] == 20140930]['S_DQ_ADJCLOSE'])

In [12]:
# create an empty list to store return for individual company
com_qtr_return = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [13]:
# Calculate quarterly return
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    com_qtr_return[i] = (end_close[i] - start_close[i])/start_close[i]

#### b) Last Quarterly Return for CSI300 Index

In [17]:
csi300_IndexData= pd.read_csv("/Users/yiliwang/Documents/ACR Internship/csi_300_IndexData_daily_.csv", header=0)

In [18]:
index_start = float(csi300_IndexData[csi300_IndexData['date'] == '2014-06-30']['last_price']) # index price on start day
index_end = float(csi300_IndexData[csi300_IndexData['date'] == '2014-09-30']['last_price'])  # index price on end day

In [19]:
index_qtr_return = (index_end - index_start)/index_start # calculate the index quarterly return

In [20]:
print index_qtr_return

0.131501358476


#### Difference = Company Return - Index Return 
Result List: norm_diff_csi300_LAST

In [22]:
diff_return_com_csi300 = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [23]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    diff_return_com_csi300[i] = com_qtr_return[i] - index_qtr_return

In [27]:
# convert into a dataframe
Weighted_Return = pd.DataFrame({'Company':BEME_ratio_df_2014['WINDCODE'],'diff_csi300_LAST': diff_return_com_csi300})

In [26]:
# Normalize Function
def normalize_list(input_list):
    empty_list = [None]*len(input_list)
    for i in range(len(input_list)):
        empty_list[i] = (input_list[i]-input_list.mean())/input_list.std()
    return empty_list

In [29]:
norm_diff_csi300_LAST = normalize_list(Weighted_Return['diff_csi300_LAST'])

###### X2: norm_diff_csi300_LAST

In [31]:
norm_diff_csi300_LAST[0:10] # preview

[-1.0777214369617809,
 0.31444535401573076,
 -0.69929169938716784,
 -0.49593344219372876,
 -0.7913283212128851,
 -1.364737261248272,
 -0.51709908433375384,
 0.0091953187722201308,
 0.067746642762259252,
 -0.20761442412704131]

### 3. AShareEODDerivativeIndicator['FLOAT_A_SHR_TODAY']
total free-float shares per day * unadjusted close price

In [36]:
# Extract trading dates of certain period 
dates_trade = frames['000001.SZ']['TRADE_DT'].unique()
dates_trade.sort() # sort the date

In [37]:
start_dateind = list(dates_trade).index(20140630) # index of start date
end_dateind = list(dates_trade).index(20140930) # index of end date

In [38]:
# target_dates: list of trading dates in that period
target_dates = dates_trade[start_dateind:end_dateind+1]

In [39]:
target_dates

array([20140630, 20140701, 20140702, 20140703, 20140704, 20140707,
       20140708, 20140709, 20140710, 20140711, 20140714, 20140715,
       20140716, 20140717, 20140718, 20140721, 20140722, 20140723,
       20140724, 20140725, 20140728, 20140729, 20140730, 20140731,
       20140801, 20140804, 20140805, 20140806, 20140807, 20140808,
       20140811, 20140812, 20140813, 20140814, 20140815, 20140818,
       20140819, 20140820, 20140821, 20140822, 20140825, 20140826,
       20140827, 20140828, 20140829, 20140901, 20140902, 20140903,
       20140904, 20140905, 20140909, 20140910, 20140911, 20140912,
       20140915, 20140916, 20140917, 20140918, 20140919, 20140922,
       20140923, 20140924, 20140925, 20140926, 20140929, 20140930])

In [40]:
def find_FAST_mean(company): # For a given company, calculate mean of FLOAT_A_SHR_TODAY *close price (unadjusted) on trading days
    FAST_product = [None]*len(target_dates)
    
    for i in range(len(target_dates)):
        close_p = frames[company][frames[company]['TRADE_DT'] == target_dates[i]]['S_DQ_CLOSE'].values # get close price on a particular date
        
        # dataframe of the company
        df_company = AShareEODDerivativeIndicator[AShareEODDerivativeIndicator['S_INFO_WINDCODE'] ==company]
        FAST_day = float(df_company[df_company['TRADE_DT']==target_dates[i]]['FLOAT_A_SHR_TODAY']) #FLOAT_A_SHR_TODAY of a particular day
        
        FAST_product[i] = FAST_day*close_p
        
    FAST_mean = np.mean(FAST_product)
    return FAST_mean

In [42]:
FAST_mean_list = [find_FAST_mean(company) for company in BEME_ratio_df_2014['WINDCODE']]

In [68]:
FAST_mean_list[0:10]

[2475450.3060000003,
 1214769.8451515154,
 21731030.157131121,
 1744583.1138873636,
 52076357.141343422,
 4217973.8803393338,
 6026764.7918766821,
 721940.02979349985,
 5228155.3201833637,
 4182176.1268759542]

##### X3: FAST_mean_list

In [44]:
FAST_mean_df = pd.DataFrame({'Company':BEME_ratio_df_2014['WINDCODE'],'FAST_mean':FAST_mean_list})

In [67]:
FAST_mean_df[0:10]

,Company,FAST_mean
0,002475.SZ,2.475450e+06
1,002470.SZ,1.214770e+06
2,600036.SH,2.173103e+07
3,002146.SZ,1.744583e+06
4,601988.SH,5.207636e+07
5,000895.SZ,4.217974e+06
6,600048.SH,6.026765e+06
7,000718.SZ,7.219400e+05
8,002304.SZ,5.228155e+06
9,600031.SH,4.182176e+06


In [46]:
FAST_mean_df.to_csv("FAST_mean_df_2") # save it as csv

## Y : Current Quarter Return

Duration: 20140930 - 20141231

return = [SP(20141231) - SP(20140930)]/SP(20140930)

sample pool: companies in BEME_ratio_df['WINDCODE']

In [47]:
# Obatin Close Price on starting and end dates for current qtr
start_close_c = [None]*len(BEME_ratio_df_2014['WINDCODE'])
end_close_c = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [48]:
# Dataframe start_close: start_close price for 268 companies

for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    company = BEME_ratio_df_2014['WINDCODE'][i]
    start_close_c[i] =float(frames[company][frames[company]['TRADE_DT'] == 20140930]['S_DQ_ADJCLOSE'])
    end_close_c[i] = float(frames[company][frames[company]['TRADE_DT'] ==20141231]['S_DQ_ADJCLOSE'])

In [49]:
# create an empty list to store return for individual company for current qtr
com_qtr_return_CURRENT = [None]*len(BEME_ratio_df_2014['WINDCODE']) 

In [50]:
# Calculate quarterly return
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    com_qtr_return_CURRENT[i] = (end_close_c[i] - start_close_c[i])/start_close_c[i]

In [52]:
index_start_c = float(csi300_IndexData[csi300_IndexData['date'] == '2014-09-30']['last_price']) 
index_end_c = float(csi300_IndexData[csi300_IndexData['date'] == '2014-12-31']['last_price']) 

In [53]:
index_qtr_return_CURRENT = (index_end_c - index_start_c)/index_start_c # calculate the index quarterly return

In [54]:
index_qtr_return_CURRENT

0.4454066480680024

In [55]:
diff_return_com_csi300_CURRENT = [None]*len(BEME_ratio_df_2014['WINDCODE'])

In [56]:
for i in range(len(BEME_ratio_df_2014['WINDCODE'])):
    diff_return_com_csi300_CURRENT[i] = com_qtr_return_CURRENT[i] - index_qtr_return_CURRENT

In [58]:
diff_return_com_csi300_CURRENT[0:10]

[-0.6002058311857696,
 -0.3644727959279245,
 0.1514835816836801,
 0.04633763643665456,
 0.08947707286223022,
 -0.5052767331450317,
 0.504187015135372,
 -0.20842551599253065,
 -0.1932311585828681,
 0.2605780685847933]

##### Y: diff_return_com_csi300_CURRENT

In [59]:
# Model Prep
Y = diff_return_com_csi300_CURRENT
X_1 = norm_BEME_ratio_list
X_2 = norm_diff_csi300_LAST
X_3 = FAST_mean_list

In [60]:
X1_matrix = np.column_stack((X_1, X_2, X_3))

In [63]:
Y1_matrix = np.array(Y)[:,np.newaxis] # matrix Y1: 268*1

In [66]:
# save as csv
pd.DataFrame(X1_matrix).to_csv("X1_test.csv") 
pd.DataFrame(Y1_matrix).to_csv("Y1_test.csv") 